# **Similarity**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [25]:
import pandas as pd
c_data = pd.read_csv('/content/gdrive/My Drive/Learning/Course/NLU/Dataset/cricket_en/cricket_dataset_short.csv', index_col=0)
c_data.head()

,description,tags
0,wankhedes notorious reputation for being a cha...,sports mumbai rahane ball bat chasing over thi...
1,pandya to rahane no run picks a short of lengt...,sports length delivery delivery leg short wris...
2,harbhajan to rahane no run pushes it quick jus...,sports shreyas gopal drifter cricket in india ...
3,shreyas gopal to pietersen leg byes one run sp...,sports cut pietersen shot off leg foot cut sho...
4,shreyas gopal to rahane no run on the stumps n...,sports foot cut ravi shaz kp pitch over off sh...


Initializations

In [26]:
%matplotlib inline
import numpy as np
import os
import pandas as pd

!pip install utils
import utils

import vsm

In [27]:
D_HOME = '/content/gdrive/My Drive/Learning/Course/NLU/Dataset'
DATA_HOME = os.path.join(D_HOME, 'vsmdata')

In [28]:
imdb5 = pd.read_csv(
    os.path.join(DATA_HOME, 'imdb_window5-scaled.csv.gz'), index_col=0)

imdb20 = pd.read_csv(
    os.path.join(DATA_HOME, 'imdb_window20-flat.csv.gz'), index_col=0)

giga5 = pd.read_csv(
    os.path.join(DATA_HOME, 'giga_window5-scaled.csv.gz'), index_col=0)

giga20 = pd.read_csv(
    os.path.join(DATA_HOME, 'giga_window20-flat.csv.gz'), index_col=0)

**Text Pre-processing**

Merge into single column

In [30]:
c_data['full_article'] = c_data['description'].str.lower() + c_data['tags'].str.lower()
c_data['full_article'].head()

0    wankhedes notorious reputation for being a cha...
1    pandya to rahane no run picks a short of lengt...
2    harbhajan to rahane no run pushes it quick jus...
3    shreyas gopal to pietersen leg byes one run sp...
4    shreyas gopal to rahane no run on the stumps n...
Name: full_article, dtype: object

Remove Punctuation

In [31]:
c_data['full_article'] = c_data['full_article'].str.replace('[^\w\s]','')

Remove Stopwords

In [32]:
import nltk
#Importing stopwords from nltk library
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
print(STOPWORDS)
# Function to remove the stopwords
def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

# Applying the stopwords to 'text_punct' and store into 'text_stop'
c_data['full_article'] = c_data['full_article'].apply(stopwords)
c_data['full_article'].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'each', 'some', 'as', 'same', 'how', "couldn't", 'needn', 'under', 'further', 'isn', "hadn't", 'and', 'am', 'just', 'doing', 'didn', "didn't", "mightn't", 'ma', "don't", "you've", 'my', 'above', 'doesn', 'ain', 'hers', 'are', 'between', 'his', 'what', "shan't", 'can', 'does', 'its', 'to', 've', "needn't", "you'll", 're', 'itself', "won't", 'yourselves', "you'd", 'here', 'theirs', 'such', 'few', 'me', 'then', 'against', 'with', 'which', "mustn't", 'has', 'own', 'mustn', 'we', "haven't", 'because', 'weren', 'so', 'him', 'who', 'their', 'again', 'had', 'on', "hasn't", 'it', 'been', 'off', 'of', 'once', 'be', 'the', 'hasn', "she's", "that'll", 'yourself', 'have', 'no', 'that', 'm', 'herself', 'd', 'now', 'haven', 'at', 'up', 'any', 'ourselves', 'very', 'after', 'those', 'was', "wouldn't", 'until', 'both', 'into', 'below', 'wouldn', 'an', 'down', 'all', "isn't", 'too', 'y'

0    wankhedes notorious reputation chasing ground ...
1    pandya rahane run picks short length delivery ...
2    harbhajan rahane run pushes quick outside skid...
3    shreyas gopal pietersen leg byes one run splay...
4    shreyas gopal rahane run stumps nudged front s...
Name: full_article, dtype: object

**Co-Occurence of Custom Dataset**

In [33]:
from collections import defaultdict
import numpy as np

def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)  # add to vocab
            next_token = text[i+1 : i+1+window_size]
            for t in next_token:
                key = tuple( sorted([t, token]) )
                d[key] += 1

    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

Remove all words with numbers

In [34]:
c_data['full_article'] = c_data['full_article'].str.replace('\d+', '')

In [61]:
df = co_occurrence(c_data['full_article'].values, 20)
df.head()

,aamir,aaron,abdomen,able,absolute,absolutely,acknowledge,across,action,actually,acutely,adapt,adds,adjudged,advances,afford,africa,african,africana,afrikaner,againp,agarwal,agarwals,ages,aggressive,agoishant,aimed,aiming,aims,aint,air,airsports,ajay,ajinkya,al,allowed,allowing,allows,almost,alone,...,widec,widejohnson,wides,widesports,widevinay,width,william,willow,winning,wins,wisden,without,wonder,wont,wood,work,worked,workrp,works,world,worse,would,wrists,wristy,wrong,wrongfooted,xi,yadav,yanks,yards,year,yes,yet,yorker,younis,zaheer,zaheers,zeroing,zone,zooms
aamir,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
aaron,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
abdomen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
able,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
absolute,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Scaled Co-occurrences of Custom Dataset**

In [62]:
from itertools import chain
from itertools import combinations
import numpy as np
def distances(sentence, window=3, scale=True):
    """Compute a dictionary mapping combinations of pairs of words in the sentence to their normalized distances.
    s1 = 'A B X B'.split()
    distances(s1, 2) -> {
        ('A', 'B'): 1.0,
        ('B', 'X'): 2.0
    }
    distances(s1, 3) -> {
        ('A', 'B'): 1.0,
        ('A', 'X'): 0.5,
        ('B', 'X'): 3.0,
        ('B', 'B'): 0.5
    }
    """
    distances = {}
    for pair in combinations(enumerate(sentence), 2):
        (i, word1), (j, word2) = pair
        word1, word2 = sorted((word1, word2))
        if (word1, word2) not in distances:
            distances[(word1, word2)] = 0.0
        distance = abs(i - j)
        if distance <= window:
            distances[(word1, word2)] += (1.0 / distance) if scale else 1.0
    return distances

def vocabulary(sentences):
    """Flatten and index the vocabulary of words in the sentences.
    
    s1 = 'A B X B'.split()
    s2 = 'X B B'.split()
    sentences = s1, s2
    vocabulary(sentences) -> {0: 'A', 1: 'B', 2: 'X'}
    """
    return dict(enumerate(set(chain(*sentences))))

In [37]:
def co_occurence_matrix(sentences, window=3, scale=True):
    
    vocab = vocabulary(sentences)
    shape = len(vocab), len(vocab)
    matrix = np.zeros(shape)
    index = {word: i for (i, word) in vocab.items()}
    for sentence in sentences:
        for (word1, word2), distance in distances(
            sentence,
            window=window,
            scale=scale
        ).items():
            cell = index[word1], index[word2]
            matrix[cell] += distance
            # maintain symmetry
            if word1 != word2:
                inverse_cell = index[word2], index[word1]
                matrix[inverse_cell] += distance
    return index, matrix

In [39]:
temp_sentences = c_data['full_article'].str.cat(sep=' ')

In [40]:
sentences = [temp_sentences.split()]
index, matrix = co_occurence_matrix(sentences, window=20, scale=True)

In [41]:
df_scaled = pd.DataFrame(matrix, columns=index)
df_scaled.insert(0, 'index', index)
df_scaled.set_index('index', inplace=True)
df_scaled.head()

,jumps,outcaughtravichandran,caughtstoinis,fell,early,regionu,saha,rohit,manladda,stopped,kohli,bowlercoulternile,ballsports,ramp,legchawla,closer,punches,higher,dances,pandyas,using,shortleg,textbook,padpardeep,actually,still,shotharbhajan,drive,wasnt,trekking,dived,punjab,leaping,singlemohit,ribcage,bracejohnson,crossseamer,curl,shows,attackcricket,...,endmurugan,drifter,able,legb,scalp,like,miscues,cannot,batc,pk,people,price,puts,driveishant,maxwell,sans,lover,aiming,misfields,finchsandeep,marillier,stopsmurugan,onto,rushed,misdirected,dishes,latecut,cricket,bring,whatsoever,much,seems,similar,ended,himfinch,ugly,lofts,legsandeep,bowler,coveraxar
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
jumps,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.090909,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
outcaughtravichandran,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.055556,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.05,0.0
caughtstoinis,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
fell,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.142857,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
early,0.0,0.0,0.0,0.0,0.0,0.0,2.156443,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.000000,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.00,0.0


**Save for further use**

In [43]:
import pickle

SAVE_HOME = os.path.join(D_HOME, 'cricket_en')
df.to_pickle(os.path.join(SAVE_HOME,'cricket20Short.pkl'))
df_scaled.to_pickle(os.path.join(SAVE_HOME,'cricket20ShortScaled.pkl'))

# **Word Comparison Start**

Non-Normalized

Eucledian Distance 

In [65]:
neighbor = vsm.neighbors('cricket', df, distfunc=vsm.euclidean)
neighbor.head()

cricket      0.000000
short      195.923454
deep       202.953197
square     221.393315
bat        224.539529
dtype: float64

In [66]:
neighbor['bat']

224.53952881397075

Cosine Distance

In [69]:
neighbor = vsm.neighbors('cricket', df, distfunc=vsm.cosine)
neighbor.head()

back       -0.813147
delivery   -0.324571
outside    -0.281261
square     -0.070965
run        -0.027998
dtype: float64

In [70]:
neighbor['bat']

0.14702027052106625

Normalization

In [73]:
df_normed = df.apply(vsm.length_norm, axis=1)
df_normed.head()

/content/vsm.py:22: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(u.dot(u))


,aamir,aaron,abdomen,able,absolute,absolutely,acknowledge,across,action,actually,acutely,adapt,adds,adjudged,advances,afford,africa,african,africana,afrikaner,againp,agarwal,agarwals,ages,aggressive,agoishant,aimed,aiming,aims,aint,air,airsports,ajay,ajinkya,al,allowed,allowing,allows,almost,alone,...,widec,widejohnson,wides,widesports,widevinay,width,william,willow,winning,wins,wisden,without,wonder,wont,wood,work,worked,workrp,works,world,worse,would,wrists,wristy,wrong,wrongfooted,xi,yadav,yanks,yards,year,yes,yet,yorker,younis,zaheer,zaheers,zeroing,zone,zooms
aamir,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.133631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.133631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaron,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.11547,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abdomen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.141421,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
able,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
absolute,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.141421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
df_normed.isnull().values.any()

True

Eucledian Distance Normalized

In [78]:
neighbor = vsm.neighbors('bat', df_normed.dropna(), distfunc=vsm.euclidean)
neighbor.head()

bat      0.000000
short    1.026365
wide     1.081168
line     1.152090
time     1.163500
dtype: float64

Cosine Distance Normalized

In [79]:
neighbor = vsm.neighbors('bat', df_normed.dropna(), distfunc=vsm.cosine)
neighbor.head()

bat      0.000000
short    0.099231
back     0.103706
wide     0.114155
line     0.118294
dtype: float64

Jaccard Distance Normalized

In [80]:
neighbor = vsm.neighbors('bat', df_normed.dropna(), distfunc=vsm.jaccard)
neighbor.head()

bat       0.000000
short     0.522826
wide      0.558240
full      0.574675
length    0.605751
dtype: float64

Matching Distance Normalized

In [81]:
neighbor = vsm.neighbors('bat', df_normed.dropna(), distfunc=vsm.matching)
neighbor.head()

subtle        0.645375
fourp         0.672970
boasts        0.684836
supergiant    0.709371
star          0.711735
dtype: float64

# **Matrix Re-weighting**

**Observed-over-expected Normalization**

In [82]:
df_scaled_oe = vsm.observed_over_expected(df_scaled)

In [83]:
neighbor = vsm.neighbors('bat', df_scaled_oe.dropna(), distfunc=vsm.cosine)
neighbor.head()

index
bat       0.000000
handed    0.592761
comes     0.688550
run       0.737454
ball      0.748569
dtype: float64

In [84]:
neighbor = vsm.neighbors('bat', df_scaled_oe.dropna(), distfunc=vsm.jaccard)
neighbor.head()

index
bat         0.000000
ball        0.810674
run         0.827343
delivery    0.834762
length      0.839623
dtype: float64

**Pointwise Mutual Information (PMI)**

In [85]:
df_scaled_pmi = vsm.pmi(df_scaled)

In [87]:
neighbor = vsm.neighbors('bat', df_scaled_pmi.dropna(), distfunc=vsm.cosine)
neighbor.head()

index
bat       0.000000
handed    0.564586
comes     0.632020
right     0.680088
face      0.720914
dtype: float64

In [86]:
neighbor = vsm.neighbors('bat', df_scaled_pmi.dropna(), distfunc=vsm.jaccard)
neighbor.head()

index
bat       0.000000
handed    0.829674
comes     0.838101
right     0.861352
caught    0.883896
dtype: float64